In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd

<IPython.core.display.Javascript object>

In [3]:
def get_vocab(vocab_path="data/raw/german_english.csv", list_columns=None):
    vocab = pd.read_csv(vocab_path)

    if list_columns is None:
        list_columns = ["id_vocab", "german", "english"]
    vocab = vocab[list_columns]

    return vocab

<IPython.core.display.Javascript object>

In [4]:
vocab = get_vocab("data/raw/german_english__feature.csv")
vocab

,id_vocab,german,english
0,1,rennen,to sprint
1,2,richtig,correct
2,3,September,September
3,4,sprechen,to speak
4,5,stimmen,to vote
5,6,verheiratet,married
6,7,wirklich,really
7,8,die Aprikose,the apricot
8,9,das Jahr,the year
9,10,viel,a lot


<IPython.core.display.Javascript object>

# Overall

In [5]:
from src.data.make_vocab_features import create_vocab_features

<IPython.core.display.Javascript object>

In [6]:
vocab_test = create_vocab_features(vocab)
vocab_test

,id_vocab,levenshtein_distance_german_english,nb_characters_german,nb_characters_english,nb_words_german,nb_words_english,is_noun,is_verb,difficulty_category
0,1,6,6,9,1,1,False,True,4.0
1,2,7,7,7,1,1,False,False,4.0
2,2,7,7,7,1,1,False,False,5.0
3,3,0,9,9,1,1,False,False,4.0
4,4,5,8,8,1,1,False,True,4.0
5,4,5,8,8,1,1,False,True,5.0
6,5,6,7,7,1,1,False,True,4.0
7,6,8,11,7,1,1,False,False,3.0
8,6,8,11,7,1,1,False,False,5.0
9,7,7,8,6,1,1,False,False,4.0


<IPython.core.display.Javascript object>

# Levenshtein distance between words

In [ ]:
def remove_article(
    vocab,
    list_german_article = ['der','die','das'],
    list_english_article = ['the','to']
):

    vocab['german'] = vocab['german'].map(
        lambda x: ' '.join(word for word in x.split(' ') if word not in list_german_article)
    )
    vocab['english'] = vocab['english'].map(
        lambda x: ' '.join(word for word in x.split(' ') if word not in list_english_article)
    )

In [ ]:
def add_levenshtein_distance(vocab):
    
    from Levenshtein import distance
    
    vocab = vocab.copy()
    
    # Lowercase
    vocab['german'] = vocab['german'].str.lower()
    vocab['english'] = vocab['english'].str.lower()
    
    # Remove article
    remove_article(vocab)
    
    # Calculate Levenshtein distance
    levenshtein_dist = vocab.apply(lambda x: distance(x['german'], x['english']), axis=1)
    
    return levenshtein_dist

In [ ]:
vocab['levenshtein_dist'] = add_levenshtein_distance(vocab)
vocab

In [ ]:
# use dill - works the same way as pickle
import dill
with open('data/processed/vocab.pkl', 'wb') as file:
    dill.dump(vocab, file)

##### Tests

In [ ]:
def assert_vocab_remove_articles():

    # Creating test dataframe
    vocab_test = pd.DataFrame(columns=['german', 'english'])
    vocab_test = vocab_test.append({'german': 'dienstag', 'english': 'tuesday'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'studieren', 'english': 'to study'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'die angst', 'english': 'the fear'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'andere', 'english': 'other'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'der arzt', 'english': 'the doctor'}, ignore_index=True)
    vocab_test = vocab_test.append({'german': 'das café', 'english': 'the café'}, ignore_index=True)

    # Creating test dataframe
    vocab_results = pd.DataFrame(columns=['german', 'english'])
    vocab_results = vocab_results.append({'german': 'dienstag', 'english': 'tuesday'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'studieren', 'english': 'study'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'angst', 'english': 'fear'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'andere', 'english': 'other'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'arzt', 'english': 'doctor'}, ignore_index=True)
    vocab_results = vocab_results.append({'german': 'café', 'english': 'café'}, ignore_index=True)

    return vocab_test, vocab_results

In [ ]:
assert_vocab_remove_articles()

# Length words

In [ ]:
vocab["nb_characters_german"] = vocab["german"].map(len)
vocab["nb_characters_english"] = vocab["english"].map(len)
vocab

In [ ]:
def count_nb_words_german(x):
    list_german_article = ["der", "die", "das"]
    separate_words = x.split(" ")
    if separate_words[0] in list_german_article:
        separate_words = separate_words[1:]
    return len(separate_words)


def count_nb_words_english(x):
    list_english_article = ["the", "to"]
    separate_words = x.split(" ")
    if separate_words[0] in list_english_article:
        separate_words = separate_words[1:]
    return len(separate_words)

In [ ]:
vocab["nb_words_german"] = vocab["german"].map(count_nb_words_german)
vocab["nb_words_english"] = vocab["english"].map(count_nb_words_english)
vocab

In [ ]:
vocab["nb_words_english"] = vocab["english"].map(lambda x: len(x.split(" ")))
vocab

In [ ]:
def is_noun(x, list_german_article=["der", "die", "das"]):
    possible_article = x["german"].split(" ", 1)[0]
    return possible_article in list_german_article


vocab["is_noun"] = vocab.apply(is_noun, axis=1)
vocab

In [ ]:
def is_verb(x):
    possible_article = x["english"].split(" ", 1)[0]
    return "to" in possible_article


vocab["is_verb"] = vocab.apply(is_verb, axis=1)
vocab

# Word difficulty category

In [ ]:
def add_difficulty_category(vocab):

    dict_difficulty_category = {
        "Minus10points": -10,
        "Minus9points": -9,
        "Minus8points": -8,
        "Minus7points": -7,
        "Minus6points": -6,
        "Minus5points": -5,
        "Minus4points": -4,
        "Minus3points": -3,
        "Minus2points": -2,
        "Minus1points": -1,
        "0points": 0,
        "1points": 1,
        "2points": 2,
        "3points": 3,
        "4points": 4,
        "5points": 5,
    }

    original_vocab = pd.DataFrame()

    for difficulty_category in dict_difficulty_category.keys():

        i_original_vocab = pd.read_csv(
            f"data/raw/new_vocabulary/{difficulty_category}.csv"
        )
        i_original_vocab["difficulty_category"] = dict_difficulty_category[
            difficulty_category
        ]
        original_vocab = original_vocab.append(i_original_vocab)

    vocab = pd.merge(
        vocab,
        original_vocab[["German", "English", "difficulty_category"]],
        left_on=["german", "english"],
        right_on=["German", "English"],
        how="left",
    )

    del vocab["German"]
    del vocab["English"]

    return vocab

In [ ]:
add_difficulty_category(vocab)